In [408]:
import random
import pandas as pd
import numpy as np
import copy
import matplotlib as plt

"Hello world"

'Hello world'

# Data Models


In [604]:


# =======================  INSERT STUDENTS INTO DATAFRAME
student_file = pd.read_csv('studentNames.csv', header=None)
student_file.reset_index(drop=True, inplace=True)
print(student_file)

print ("THIS IS ONE STUDENT:", student_file[0][1])

# =======================  INSERT COURSES INTO DATAFRAME
course_file = pd.read_csv('courses.csv', header=None)
print(course_file)


# =======================  INSERT STUDENT COURSES INTO DATAFRAME
student_course_file = pd.read_csv('studentCourse.csv')
print(student_course_file)


# =======================  INSERT TEACHERS INTO DATAFRAME
teacher_file = pd.read_csv('teachers.csv')
print(teacher_file)

# =======================  INSERT CLASSROOMS INTO DATAFRAME



# =======================  INSERT DAYS INTO DATAFRAME





## Construct a valid input here (according to what we need)

# Structure (Bottom-Up):
#        1. Gene : Contains 1 course, 1 instructor , and an array of Students, It's corresponding Day (mon, tue, wed ...) ,
#                  It's corresponding Time (9-12, 10-1 ...).
#        2. Chromosome: Contains a classroom , TimeTable for the classroom , fitness value of the generated Timetable
#        3. Member: Contains a list of chromosomes
#        4. Population:  Contains a list of members


# initialise population 

Classrooms_Array = ['c301','c302','c303','c304','c305','c306','c307','c308','c309','c310']
# Courses_Array = ['CS217','CS218','CS219','CS220','CS221','CS222']
Courses_Array = list(course_file[0])


Instructors_Array = ['Ayesha Bano','Aqeel Shahzad','Usman Rashid','Maimoona Rassol','Muhammad bin Qasim','Sanaa Ilyas']

# Students_Array = ['Bano','Aqeel Usman ','Shahzad Rashid','Maimoona Qasim','Muhammad bin Aqeel','Sanaa Rassol']

Students_Array = list(student_file[0])

print('Printing students array')
print(Students_Array)

Days_Array = [
#               'Sunday', 
              'Monday',
              'Tuesday',
              'Wednesday',
              'Thursday',
              'Friday',
#               'Saturday',
            ]

class Course:
    def __init__ (self, id, name):
        self.id = id
        self.name = name

class Instructor:
    def __init__ (self, name):
        self.name = name
        
class Student:
    def __init__ (self, name):
        self.name = name

class Classroom:
    def __init__ (self, id, capacity):
        self.id = id
        self.capacity = capacity
        
class Student_Course:
    def __init__ (self, course, student):
        self.course = course
        self.student_id = student


                     0
0        Mohammed Azam
1    Mohamed F Bakaram
2      Manal Aldaihani
3        Ibrar Manawer
4         Adam Flowers
..                 ...
195     Naveed Hussain
196      Sarah L Clark
197       Maria Lytras
198        Bibi A Khan
199       Iram Matloob

[200 rows x 1 columns]
THIS IS ONE STUDENT: Mohamed F Bakaram
         0                                            1
0    CS217                  Object Oriented Programming
1    EE227                         Digital Logic Design
2    CS211                          Discrete Structures
3    SE110                Intro to Software Engineering
4    CS118                     Programming Fundamentals
5    CS219                             Database Systems
6    CS220                            Operating Systems
7    CS302              Design & Analysis of Algorithms
8   CY2012                            Digital Forensics
9    CS307                            Computer Networks
10   CS328                         Software E

# Population Model

In [614]:
class Gene:
    def __init__(
            self,
            course=None,
            instructor=None,
            students=None,
            day=None,
            start_time=None,
            end_time=None,
            classroom_name=None,
    ):

        if course is None:
            course = Courses_Array[random.randint(0, len(Courses_Array) - 1)]

        if instructor is None:
            instructor = Instructors_Array[random.randint(0, len(Instructors_Array) - 1)]

        if students is None:
            std_list = []
            for i in range (1, 30):
                L = random.randint(1, len(Students_Array)-1)
                std_list.append(L)
            students = np.array(std_list)
#             print ("STUDENTS ARE", students)

        if day is None:
            day = Days_Array[random.randint(0, len(Days_Array) - 1)]

        if start_time is None:
            start_time = random.randint(9.0, 17.0 - 3)
        
        if end_time is None:
            ## Logical way is to start this range from start_time
            end_time = start_time+3

        if classroom_name is None:
            classroom_name = Classrooms_Array[random.randint(0, len(Classrooms_Array) - 1)]
        
        self.course = course
        self.instructor = instructor
        self.students = students
        self.day = day
        self.start_time = start_time
        self.end_time = end_time
        self.classroom_name = classroom_name

    def __str__(self):
        
        student_names = []
        for student in self.students:
            student_names.append(Students_Array[int(student)])
                                
        return (
                self.classroom_name  + "\t" +
                self.day             + "\t" +
                str(self.start_time) + "\t" +
                str(self.end_time)   + "\t" +
                self.course          + "\t" +
                self.instructor      + "\t" +
                str(student_names)        + "\t" +
                "."
               )

class Chromosome:

    def __init__(self,
                 genes=None,
                 fitness_value=None,
                 ):


        if genes is None:
            genes = []
            for i in range(0, random.randint(30, 50)):
                genes.append(Gene())

        
        if fitness_value is None:
            fitness_value=0
        
        
        self.genes = np.array(genes)
       
        self.fitness_value = fitness_value
        
    def __getitem__(self, index):
        return self.genes[index]

class Population:

    def __init__(self,
                 chromosomes=None
                 ):

        if chromosomes is None:
            chromosomes = []
            for i in range(1, 20):
                chromosomes.append(Chromosome())

        self.chromosomes = np.array(chromosomes)                
                
    def __getitem__(self, index):
        return self.chromosomes[index]
    

# Initialization

In [602]:
def initialize_population():
    
    new_population = Population()

    
#     ----------------------------------------------- Debugging
#     for member in new_population.members:
#         for chromosome in member.chromosomes:
#             for gene in chromosome.genes:
#                 print (gene.course,gene.instructor,gene.students,gene.day,gene.start_time,gene.end_time)
#     ----------------------------------------------- Debugging


    return new_population

In [603]:
new_pop = initialize_population()

for chromosome in new_pop:
#         print("\n\nCLASSROOM:", chromosome.classroom_name)
    sort_by_day = sorted(chromosome, key=lambda gene: (gene.day, gene.start_time) )
    for gene in sort_by_day:
        print(gene)

c301	Friday	9	12	MT224	Maimoona Rassol	['Parastoo Jamshidi', 'Iram Matloob', 'Huda O Ali', 'Halima Sayed', 'Negar Saleh', 'Ali M Sadeghioon', 'Sarah R Young', 'Adam Selby', 'Nima Dirie', 'Omar F Khan', 'Sarah Austin', 'Zahir Hussain', 'Nima Dirie', 'Omar Chaudhrey', 'Seema Mandalia', 'Sarina Patel', 'Razan Z Haji Zulkifli', 'Meryem Mehmetbasoglu', 'Safiya Issa', 'Huda O Ali', 'Yas Khaled', 'Shireen Dowlut-Beebeejaun', 'Sara De La Serna', 'Sarah Nolasco', 'Lilia Kyuchukova', 'Maria Lytras', 'Yasmin Ahmed', 'Sheila Hughton', 'Sadia Rehman']	.
c307	Friday	12	15	CS328	Usman Rashid	['Zeenat K Guness-Goolbar', 'Maria Antonia Morua Leyva', 'Anita Hodler', 'Sadia Rehman', 'Omar F Khan', 'Zeenat K Guness-Goolbar', 'Sarah Hadley', 'Mohamed F Bakaram', 'Salma Khanam', 'Sarah L Barber-Jones', 'Ana Vukojevic', 'Mohamed A Baalousha', 'Reem N Hassan', 'Sarah J Hammon', 'Imtiaz Kermali', 'Natasha Edwards', 'Natasha J Seaward', 'Adam Sanders', 'Samar E Nassar', 'Ali M Sadeghioon', 'Sarah Taylor', 'Kabi

# Selection

In [545]:

def roulette_wheel_selection(population):
    #     print ("I'm in roulette wheel!")

    maximum_val = sum(chromosome.fitness_value for chromosome in population)
    pick = random.uniform(0, maximum_val)
    current = 0

    mom, dad = None, None

    for chromosome in population:
        current += chromosome.fitness_value
        if current > pick:
            mom = chromosome
            break

    pick = random.uniform(0, maximum_val)
    current = 0
    for chromosome in population:
        current += chromosome.fitness_value
        if current > pick:
            dad = chromosome
            break

    if mom is None:
        mom = population[random.randint(0,len(population.chromosomes)-1)]

    if dad is None:
        dad = population[random.randint(0, len(population.chromosomes) - 1)]

    return mom, dad


def elitism(population):
    max_index_1 = 0
    max_index_2 = 0
    max_fitness1 = 0
    max_fitness2 = 0

    for idx, individual in enumerate(population.chromosomes):
        if individual.fitness_value > max_fitness2:
            if individual.fitness_value > max_fitness1:
                max_fitness2 = max_fitness1
                max_index_2 = max_index_1
                max_fitness1 = individual.fitness_value
                max_index_1 = idx
            elif individual.fitness_value > max_fitness2:
                max_fitness2 = individual.fitness_value
                max_index_2 = idx

    return population[max_index_1], population[max_index_2]


def select_best_mom_and_dad(population):
    ## Since we're instructed to do Roulette_wheel_selection...
#     return roulette_wheel_selection(population)

    ## i g n o r e d
    return elitism(population)



In [546]:
new_pop = initialize_population()

for member in new_pop:
    member.fitness_value = random.randint(0,20)

mom,dad = select_best_mom_and_dad(new_pop)

print(" M O M ")

for gene in mom:
    print(gene)
        
print()
print(" D A D ") 

for gene in dad:
    print(gene)

 M O M 
c309	Saturday	11	13	CS211	Usman Rashid	['Maria A Grenfell', 'Mohammed Riyaz Shahul Hameed Mohammed Yakub', 'Iram Meharban', 'Behzad Babakhani', 'Ibrahim Adegboye', 'Yaser A Aljammaz', 'Nadine Meyer', 'Mohamed A Baalousha', 'Adam Mickiewicz', 'Maria Antonia Morua Leyva', 'Halima Sayed', 'Ali M Sadeghioon', 'Adam N Starling', 'Zahra Faraji Rad', 'Sam D Edwards', 'Anja Brezovnik', 'Aleena Unadkat', 'Aida Hoxha', 'Eileen Haycox', 'Parastoo Jamshidi', 'Faris Quaid', 'Syed M Abbas', 'Ayse Hatunoglu', 'Meryem Mehmetbasoglu', 'Nima Dirie', 'Samina Akbar', 'Natasha Leeson', 'Salma Khanam', 'Sarah Wilson']	.
c302	Monday	12	15	AI2011	Aqeel Shahzad	['Sarah Fogg', 'Irfan Yousaf', 'Ayse Hatunoglu', 'Sarah J Roberts', 'Ayda Niazi', 'Sam Isherwood', 'Sarina Patel', 'Farzana Yamin', 'Sarah Taylor', 'Sarah Hinett', 'Asim Ali', 'Tina Nasersharif', 'Mohammed Sukkari', 'Sarah E Kirkham-Slack', 'Shahina Parveen', 'Maria A Grenfell', 'Maria M Ponce Carpio', 'Shireen Dowlut-Beebeejaun', 'Adam C Ryan',

# Crossover

In [577]:
def binary_crossover(husband, wife):
    length = len(husband.genes) // 2
    children = []
    son = husband.genes[:length] + wife.genes[length:]
    daughter = wife.genes[:length] + husband.genes[length:]

    children.append(son)
    children.append(daughter)

    return children


def randomized_fixed_point_crossover(mom, dad):
    # Random point must be between the length of the best member
#     print("Length of member 1 are:", len(chromosome_1.genes))
    pass_loop = True

    new_pop = []

    while pass_loop:
        try:
            random_fixed_point = random.randint(1, len(mom.genes) - 1)
            son      = np.concatenate( (mom.genes[:random_fixed_point], dad.genes[random_fixed_point:]) )
            daughter = np.concatenate( (dad.genes[:random_fixed_point], mom.genes[random_fixed_point:]) )

            new_pop.append(son)
            new_pop.append(daughter)

#             for gene_a, gene_b in zip(chromosome_a, chromosome_b):
#                 if (gene_a == gene_b):
#                     print ("Gene is same!")
            
#             if chromosome_a == chromosome_b:
#                 print ("Chromosome is same!")
            
            pass_loop = False
        except Exception as e:
            print("Error! ", str(e))
            pass_loop = True

    return new_pop


def crossover(mom, dad, population):
    #     return binary_crossover(mom, dad)
    new_chromosomes = []
    for i in range(0, random.randint(2, 8)):
        children = randomized_fixed_point_crossover(mom, dad)
        for child in children:
#             if (child not in new_chromosomes):
            new_chromosomes.append(Chromosome(genes=child))

    return Population(chromosomes=new_chromosomes)


# Constraints

In [605]:

def hard_test_invigilator(chromosome):
    wrong_value = 0
    wrong_genes = []

    # for gene1 in chromosome.genes:
    #     for gene2 in chromosome.genes:
    #         if gene1.day == gene2.day:
    #             if gene1.start_time == gene2.start_time:
    #                 wrong_genes.append(gene1)
    #                 wrong_genes.append(gene2)
    #                 wrong_value += 1

    i = 0
    while i < len(chromosome.genes) - 1:
        j = i
        while j < len(chromosome.genes) - 1:
            j += 1
            if chromosome[i].day == chromosome[j].day:
                if chromosome[i].instructor == chromosome[j].instructor:
                    if abs(chromosome[i].start_time - chromosome[j].start_time) < 3:
                        if (chromosome[i], chromosome[j]) not in wrong_genes and (
                                chromosome[j], chromosome[i]) not in wrong_genes:
                            # wrong_genes.append((chromosome[i], chromosome[j]))
                            # wrong_value += 1
                            return False
        i += 1

    for genes in wrong_genes:
        print(genes[0])
        print(genes[1])

    # return wrong_value
    return True


def hard_test_valid_paper_duration(chromosome):
    #     for chromosome in chromosomes:
    points = 0
    for gene_p in chromosome:
        if (gene_p.end_time - gene_p.start_time) != 3:
            continue
        else:
            points += 10

    #     print("I am true at: ",gene_p.start_time,gene_p.end_time)
    return points


def hard_test_no_exam_on_weekends(chromosome):
    #     for chromosome in chromosomes:
    points = 0
    for gene_p in chromosome:
        if (gene_p.day in ["Saturday", "Sunday"]):
            continue
        else:
            points += 10
    return points




# def hard_test_one_invigilator_per_exam(chromosome):
#     """At a given time, one instructor can only invigilate one exam"""
#     pass



## Possible generic constraints
def hard_test_student_one_exam_at_a_time(chromosome):
    """At a given time, student can only give one exam"""
    # Note: Also test the current gene from which the student is selected, because the same student might be repeated in a single class
    points = 0
    i = j = 0

    while i < len(chromosome.genes):
        for student in chromosome.genes[i].students:
            for genes in chromosome:
                if genes == chromosome.genes[i]:
                    continue
                if genes.start_time == chromosome.genes[i].start_time and genes.day == chromosome.genes[i].day:
                    if student in genes.students:
                        # print('Student in multiple exam')
                        continue
                    else:
                        points += 10
        i += 1

    return points

def hard_test_one_exam_per_course(chromosome):
    """First hard constraint: Every course must have an exam"""
    pass

def hard_test_one_exam_per_classroom(chromosome):
    """At a given time, a classroom can only have one exam ongoing"""
    pass

def hard_test_classroom_capacity(chromosome):
    """Number of students in an exam must not exceed the classroom capacity"""
    pass

def hard_test_students_taking_correct_exam(chromosome):
    """Students must be taking the exam of the course they're registered for"""
#     print(student_course_file)
    for genes in chromosome:
        for student in genes.students:
#             print(student_course_file["StudentName"])
            mask = student_course_file["StudentName"] == student
            student_courses = student_course_file[mask]["CourseCode"]
#             print(student_courses)
            if genes.course not in student_courses:
                return False
            
    return True


def soft_test_consecutive_exams(data):
    print("Hello soft constraint test")
    return True


HARD_CONSTRAINTS = [
    {
        "name": "Paper must be of 3 hours",
        "func": hard_test_valid_paper_duration,
        "fields": [
            'start_time',
            'end_time',
        ]
    },
#     {
#             "name":"Instructor cannot invigilate two consecutive exams",
#             "func":hard_test_invigilator,
#             "fields":[
#                 'instructor',
#             ]
#     },
    {
        "name": "No exams on weekend",
        "func": hard_test_no_exam_on_weekends,
        "fields": [
            'day',
        ]
    },
    {
        "name": "One exam at a time",
        "func": hard_test_student_one_exam_at_a_time,
        "fields": [
            'students',
        ]
    },
#     {
#         "name": "Student taking correct exam ",
#         "func": hard_test_students_taking_correct_exam,
#         "fields": [
#             'students',
#         ]
#     },
]

soft_constraints = [
    {
        "name": "Student should not have consecutive exams",
        "func": soft_test_consecutive_exams
    },
    {
        "name": "Student should not have consecutive exams",
        "func": soft_test_consecutive_exams
    },
    {
        "name": "Student should not have consecutive exams",
        "func": soft_test_consecutive_exams
    },
    {
        "name": "Student should not have consecutive exams",
        "func": soft_test_consecutive_exams
    },

]


# Fitness of chromosome

In [606]:

def set_fitness(chromosome_p):
    constraints_passed = 0
    fields_to_mutate = []

    # Right now, checking only hard constraints
    for constraint in HARD_CONSTRAINTS:
        constraints_passed += constraint["func"](chromosome_p)
        if constraints_passed < 1000:
            if constraint["fields"] not in fields_to_mutate: fields_to_mutate += constraint["fields"]

    # Set fitness value of individual chromosome
    chromosome_p.fitness_value = constraints_passed

    return constraints_passed, fields_to_mutate


In [607]:
new_pop = initialize_population()

for chromosome in new_pop:
    chromosome.fitness_value = random.randint(0,20)

mom, dad = select_best_mom_and_dad(new_pop)

fields_to_mutate = []


fitness_value, temp_fields = set_fitness(mom)
if temp_fields not in fields_to_mutate: fields_to_mutate.append(temp_fields)
else: fields_to_mutate.append([])
    
        
print(fields_to_mutate)

[['start_time', 'end_time', 'day']]


# Mutation

In [608]:

def modify_gene(gene, fields):
    # Two different approaches:
    # 1. Only mutate those values which were found to violate the constraints e.g time was not right so mutate time only
    #    -> will require a list of all constraints not met by the chromosome
    # 2. Mutate the entire gene completely

    # For now, let's try a combination of both.
    # Our model is partially self-aware.
    # So what we can do is that flip a coin.
    # If heads, then return a completely new gene.
    # If tails, apply some self-awareness.

    coin = random.randint(0, 1)

    # First make a completely random new Gene
    new_gene = Gene()

    # If heads...
    if coin:
        return new_gene

    # Else if tails...

    # Modify the existing fields of the selected gene
    for field in fields:
        setattr(gene, field, getattr(new_gene, field))

    return gene


def modify_chromosome(chromosome, random_index):
    
    # if heads, delete a gene
    # if tails, regenerate the entire chromosome
    coin = random.randint(0, 1)

    if coin:
        np.delete(chromosome.genes, random_index)
    else:
        chromosome = Chromosome()

    return chromosome

def mutate(chromosome, fields_to_mutate):
#     print("Chromosome length:", len(chromosome.genes))
    random_index = random.randint(0, len(chromosome.genes) - 1)
    
    coin = random.randint(0, 1)
    
    # if heads, modify a gene
    if coin:
        chromosome.genes[random_index] = modify_gene(chromosome.genes[random_index], fields_to_mutate)
    # if tails, modify the chromosome itself
    else:
        chromosome = modify_chromosome(chromosome, random_index)

#     print ("Chromosome is:", chromosome)

    return chromosome



In [615]:

    new_pop = initialize_population()

    for chromosome in new_pop:
        chromosome.fitness_value = random.randint(0,20)
    
    mom, dad = select_best_mom_and_dad(new_pop)

    fields_to_mutate = []

    fitness_value, temp_fields = set_fitness(mom)
    if temp_fields not in fields_to_mutate:
        fields_to_mutate.append(temp_fields)
    else:
        fields_to_mutate.append([])

#     for gene in mom:
#         print(gene)
        
    print(fields_to_mutate)

#     print('Mom genes before mutation')
#     for chromosome in mom:
#         for gene in chromosome:
#             print(gene)
    random_index = random.randint(0, len(mom.genes) - 1)
    previous_constraint, fields_to_mutate = set_fitness(mom)
    print('Constraints met before mutation: ', previous_constraint)

    for i in range(0, 200):
        mom = mutate(mom, fields_to_mutate)

        new_constraint, fields_to_mutate = set_fitness(mom)

    for gene in mom:
        print(gene)
        
    print('Constraints met after mutation: ', new_constraint)

[['start_time', 'end_time']]
Constraints met before mutation:  26790


KeyboardInterrupt: 

# Evaluation


In [610]:
min_soft_constraints = 3

def evaluate(candidate, tatti):
    
#     for constraint in hard_constraints:
#         if (constraint["func"](solution) == False):
#             return False

#     print (tatti)

    if tatti is None:
        tatti = copy.deepcopy(candidate)
    elif candidate.fitness_value > tatti.fitness_value:
        tatti = copy.deepcopy(candidate)
#     soft_constraint_pass_counter = 0
#     for constraint in soft_constraints:
#         if (constraint["func"](solution) == True):
#             soft_constraint_pass_counter += 1
#         if soft_constraint_pass_counter > min_soft_constraints:
#             return True
            
    
    return tatti



def print_mom_and_dad(mom, dad):
    print("Mom genes:")

    for gene in mom:
        print(gene)

    print("Mom Fitness Value: ", mom.fitness_value)
    print("\n")
    print("Dad genes:")
    
    for gene in dad:
        print(gene)
    
    print("Dad Fitness Value: ", dad.fitness_value)




In [611]:
# evaluate(mom)

# Our Genetic Algorithm

## Basic
Steps are:
- ~Generate New Population~
- ~Initial Selection of Parents~
- ~Crossover~
- ~Fitness~
- ~Mutate~
- ~Select best mom and dad~
- ~Evaluate~

## Advanced
- ~Use pandas, numpy!~
- ~Set accurate constraints (we have issues in some constraints where we are only checking Intra-chromosome, whereas we should be checking  INTER-chromosome e.g. Teacher constraint, sometimes teacher might appear in different classrooms)~ (Fixed by putting classroom_name in Gene class)
- ~Completely clean out "Member" implementations~
- ~Generate more children~
- ~Roulette wheel selection~
- ~Population regeneration if evaluation failure, if certain number of failures happen then stop~
    ~- If mom and dad better than last time, then continue. Maybe you'll find better~
- Days should have IDs, so that we can also go to the next week incase the current week is full
- Make all test cases:
    - Hard constraints
    - Soft constraints
- Generate a completely correct value



## Cherry-on-top
- Use data from the CSVs

In [621]:
def GeneticAlgorithm():

    max_generations = random.randint(20,300)
    mutation_probability = random.randint(0, 50)
    qoum_check = random.randint(10, max_generations//4)
    hidayah = random.randint(0,5)
    qiyamah = random.randint(hidayah, qoum_check//2)

    
    print ("==========================    HYPERPARAMETERS    ==========================")
    print ("Max generations\t\t:", max_generations)
    print ("Mutation Probability\t:", mutation_probability)
    print ("Qoum Check\t\t:", qoum_check)
    print ("Hidayah\t\t\t:", hidayah)
    print ("Qiyamah\t\t\t:", qiyamah)
    print ("\n\n")
    
    new_generation = initialize_population()

    solution_counter = 0
    prev_best_solution = None
    best_solution = None
    
    ## Two options:
    ## Randomize initial mom and dad

    ## Or select the best
    for chromosome in new_generation:
        set_fitness(chromosome)        
    mom, dad = select_best_mom_and_dad(new_generation)
    
    
#     print("========================= Before:")
#     print_mom_and_dad(mom, dad)

    
    print("\n\n")

    
    
    
    
    
    
    
    
    
# -------------------------------- START GA LOOP
    
    
    for i in range(0, max_generations):

#         print("Mom is = ", mom)
#         print("Dad is = ", dad)
#         print("Crossing over...")
        children = crossover(mom, dad, new_generation)
        new_chromosomes = []
        fields_to_mutate = []
        for idx, chromosome in enumerate(children):

#             print("Setting fitness...")
            fitness_value, temp_fields = set_fitness(chromosome)
            fields_to_mutate.append(temp_fields)

#             if fitness_value == -1:
#                 continue

            if random.randint(0, 100) < mutation_probability:
#                 print("Mutating...")
#                 prev_fitness_value = copy.deepcopy(fitness_value)
#                 while fitness_value <= prev_fitness_value:
                mutate(chromosome, fields_to_mutate[idx])
                fitness_value_changed, fields_to_mutate[idx] = set_fitness(chromosome)
                fitness_value = fitness_value_changed

            new_chromosomes.append(chromosome)



#         sorted_generation = sorted(new_generation, key=lambda chromosome: chromosome.fitness_value )

#         new_generation.chromosomes = sorted_generation[len(sorted_generation)//2:]
               
#         for chromosome in new_chromosomes:
#             new_generation.chromosomes.append(chromosome)

            
        next_gen = Population(chromosomes=new_chromosomes)

#         for chromosome in next_gen:
#             print("Fitness value", chromosome.fitness_value)

        mom, dad = select_best_mom_and_dad(next_gen)
        
        best_solution = evaluate(mom, best_solution)
        
        ### print Every 100th generation results
        if i % qoum_check == 0:
            
            
            print ('\nCurrent generation..: {}'.format(i))
            print ('Current Mom fitness value:', mom.fitness_value)
            print ('Current Dad fitness value:', dad.fitness_value)
            print ('Best solution so far: {}'.format(best_solution.fitness_value))
            if prev_best_solution is None:
                prev_best_solution = best_solution
            if best_solution.fitness_value > prev_best_solution.fitness_value:
                solution_counter = 0
            elif best_solution.fitness_value == prev_best_solution.fitness_value:
                solution_counter += 1
            
            print ("Muhlat counter:", solution_counter)
            
            if solution_counter > qiyamah:
                # Too much stagnation
                print('"And if you turn away, He will replace you with another people; then they will not be the likes of you." - (Qur’an, 47: 38)')
                new_generation = initialize_population()
                np.append(new_generation.chromosomes, mom)
                np.append(new_generation.chromosomes, prev_best_solution)
        
                solution_counter = 0

            elif solution_counter > hidayah:
                # Reintroduce  D A D into chromosome pool
                print('"And We made them leaders, guiding people by Our Command, and We sent them inspiration to do good deeds, to establish regular prayers, and to practice regular charity; and they constantly worshipped Us." (Qur’an, 21: 73)')
                dad = best_solution
                
            else:
                prev_best_solution = best_solution
        
        
# -------------------------------- END GA LOOP


        
        
        
        
        
        
    print("\n\n")
    
#     print("========================= After 1000 iterations:")
#     print_mom_and_dad(mom, dad)

    print ("\n\n=================================------------======================================")

    print ("Our best solution is:")
    for gene in best_solution:
        print (gene)
        
    print ("\nFitness value is: ", best_solution.fitness_value)


In [622]:
GeneticAlgorithm()

==========================    HYPERPARAMETERS    ==========================
Max generations		: 265
Mutation Probability	: 10
Qoum Check		: 62
Hidayah			: 1
Qiyamah			: 29







Current generation..: 0
Current Mom fitness value: 27330
Current Dad fitness value: 27330
Best solution so far: 27330
Muhlat counter: 1

Current generation..: 62
Current Mom fitness value: 37220
Current Dad fitness value: 37220
Best solution so far: 41440
Muhlat counter: 0

Current generation..: 124
Current Mom fitness value: 26840
Current Dad fitness value: 26840
Best solution so far: 41440
Muhlat counter: 1

Current generation..: 186
Current Mom fitness value: 30180
Current Dad fitness value: 30180
Best solution so far: 41440
Muhlat counter: 2
"And We made them leaders, guiding people by Our Command, and We sent them inspiration to do good deeds, to establish regular prayers, and to practice regular charity; and they constantly worshipped Us." (Qur’an, 21: 73)

Current generation..: 248
Current Mom fitness va